# Modify labels from COCO format to YOLO format

The format of the labels in `train.csv` are xmin ymin w h. We need to convert these to the YOLO format, which are normalized from 0 to 1 and xmin ymin w h . 

In [ ]:
from ast import literal_eval
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from google.colab import drive

drive.mount('/content/gdrive')

train_df = pd.read_csv("/content/gdrive/My Drive/wheat/input/train.csv")
train_df.head()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [ ]:
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def convert_to_yolo_label(coco_format_box, w = 1024, h = 1024):
    bbox = literal_eval(coco_format_box)
    xmin = bbox[0]
    xmax = bbox[0] + bbox[2]
    ymin = bbox[1]
    ymax = bbox[1] + bbox[3]
    b = (float(xmin), float(xmax), float(ymin), float(ymax))
    yolo_box = convert((w, h), b)
    if np.max(yolo_box) > 1 or np.min(yolo_box) < 0:
        print("BOX HAS AN ISSUE")
    return yolo_box

train_df['yolo_box'] = train_df.bbox.apply(convert_to_yolo_label)

In [ ]:
print("We have {} unique images with boxes.".format(len(train_df.image_id.unique())))
unique_img_ids = train_df.image_id.unique()

We have 3373 unique images with boxes.


In [ ]:
folder_location = "/content/gdrive/My Drive/wheat/input/wheat_yolo_train_data"
for img_id in unique_img_ids:
    filt_df = train_df.query("image_id == @img_id")
    #print(filt_df.shape[0])
    all_boxes = filt_df.yolo_box.values
    file_name = "{}/{}.txt".format(folder_location,img_id) # specify the name of the folder and get a file name

    s = "0 %s %s %s %s \n" 
    with open(file_name, 'a') as file:
        for i in all_boxes:
            new_line = (s % tuple(i))
            file.write(new_line)  

In [ ]:
import glob
all_imgs = glob.glob("/content/gdrive/My Drive/wheat/input/train/*.jpg")
all_imgs = [i.split("/")[-1].replace(".jpg", "") for i in all_imgs]
positive_imgs = train_df.image_id.unique()

In [ ]:
negative_images = set(all_imgs) - set(positive_imgs)
print(len(all_imgs), len(positive_imgs))

3422 3373


In [ ]:
for i in list(negative_images):
    file_name = "/content/gdrive/My Drive/wheat/input/wheat_yolo_train_data/{}.txt".format(i)
    with open(file_name, 'w') as fp: 
        pass

Now we will copy all of the .txt bounding boxes and the pictures into the same folder and zip it. 

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
!mkdir data
%cd data
!cp /content/gdrive/My\ Drive/wheat/input/wheat_yolo_train_data/* .
print("Done copying labels. Now copying images.")
!cp /content/gdrive/My\ Drive/wheat/input/train/*.jpg .
print("Done copying Images. Now zipping.")
!zip -r newobj.zip /content/data/* 
print("Finally done.")

Streaming output truncated to the last 5000 lines.
  adding: 4789e5c97.jpg (deflated 0%)
  adding: 4789e5c97.txt (deflated 74%)
  adding: 478e21230.jpg (deflated 1%)
  adding: 478e21230.txt (deflated 74%)
  adding: 47a1184e4.jpg (deflated 1%)
  adding: 47a1184e4.txt (deflated 15%)
  adding: 47a9b9c85.jpg (deflated 0%)
  adding: 47a9b9c85.txt (deflated 75%)
  adding: 47b0d504c.jpg (deflated 0%)
  adding: 47b0d504c.txt (deflated 68%)
  adding: 47dcd5b34.jpg (deflated 0%)
  adding: 47dcd5b34.txt (deflated 69%)
  adding: 47e218059.jpg (deflated 0%)
  adding: 47e218059.txt (deflated 76%)
  adding: 480c01244.jpg (deflated 0%)
  adding: 480c01244.txt (deflated 73%)
  adding: 48269749e.jpg (deflated 0%)
  adding: 48269749e.txt (deflated 73%)
  adding: 482d7c843.jpg (deflated 0%)
  adding: 482d7c843.txt (deflated 74%)
  adding: 483f5b46c.jpg (deflated 0%)
  adding: 483f5b46c.txt (deflated 74%)
  adding: 483fac02a.jpg (deflated 0%)
  adding: 483fac02a.txt (deflated 74%)
  adding: 484674836.jpg (